In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
#defining tickers and date interval
tickers = {'GLD':'GLD', 'DXY':'DX-Y.NYB', 'TNX':'^TNX', 'GSPC':'^GSPC'}
start_date = '2019-01-01'
end_date = '2026-01-01'

In [3]:
#loading data from yahoo!finance
def download_ticker(ticker):
    df = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
    df.to_csv(f'/Users/mihailborisov/Desktop/gold/raw data/{ticker}_data.csv', index=True, index_label='Date', encoding='utf-8')
    return df

In [4]:
#getting data
dfs = {name: download_ticker(t) for name, t in tickers.items()}

In [5]:
#dataframe overview
def information(df):
    print(df.head(5))
    print(df.tail(5))
    print("NaN values: ",df[df.isna().any(axis=1)])
    print("Duplicates: ",df.duplicated().any())
    return df.info()

In [6]:
#getting overview of dataframes
for t in dfs.keys():
    print(f'Ticker {t}')
    information(dfs[t])
    print(' ')

Ticker GLD
Price            Close        High         Low        Open    Volume
Ticker             GLD         GLD         GLD         GLD       GLD
Date                                                                
2019-01-02  121.330002  121.750000  120.879997  121.349998  12776200
2019-01-03  122.430000  122.459999  121.730003  121.779999  15440800
2019-01-04  121.440002  121.599998  120.690002  121.320000  13334000
2019-01-07  121.860001  122.220001  121.570000  122.160004   7473600
2019-01-08  121.529999  121.709999  121.059998  121.150002   7643100
Price            Close        High         Low        Open    Volume
Ticker             GLD         GLD         GLD         GLD       GLD
Date                                                                
2025-12-24  411.929993  412.630005  408.829987  412.250000   6718400
2025-12-26  416.739990  418.450012  414.750000  416.480011  10476200
2025-12-29  398.600006  403.760010  395.329987  403.660004  20679200
2025-12-30  398.890015 

In [7]:
def prepare_data(df):
    #droping ticker from index
    df.columns = df.columns.droplevel('Ticker')
    # adjusting dates
    df = df.sort_index()
    df.index = pd.to_datetime(df.index)
    #renaming columns to "Adjusted", because data was already adjusted (see !auto_adjust=True! in the 'loading data' step)
    for column in df.columns:
        if column != 'Volume':
            df.rename(columns={column: f'Adj {column}'}, inplace=True)
    return df

In [8]:
#dataframes cleaning
dfs = {k: prepare_data(v) for k,v in dfs.items()}

In [9]:
#calculating some metrics
def metrics_calculate(df):
    df['Return'] = df['Adj Close'].pct_change()
    df['LogReturn'] = np.log(df['Adj Close']).diff()
    df['RollVol30'] = df['Return'].rolling(30).std() * np.sqrt(252)
    df['RollMean30'] = df['Return'].rolling(30).mean() * 252
    return df

In [10]:
#executing calculations within dataframes
dfs = {k: metrics_calculate(v) for k,v in dfs.items()}

In [26]:
#saving clean data
for t in dfs.keys():
    dfs[t].to_csv(f'/Users/mihailborisov/Desktop/gold/clean data/{t}_clean.csv', index=True, index_label='Date', encoding='utf-8')

In [11]:
dfs

{'GLD': Price        Adj Close    Adj High     Adj Low    Adj Open    Volume  \
 Date                                                                   
 2019-01-02  121.330002  121.750000  120.879997  121.349998  12776200   
 2019-01-03  122.430000  122.459999  121.730003  121.779999  15440800   
 2019-01-04  121.440002  121.599998  120.690002  121.320000  13334000   
 2019-01-07  121.860001  122.220001  121.570000  122.160004   7473600   
 2019-01-08  121.529999  121.709999  121.059998  121.150002   7643100   
 ...                ...         ...         ...         ...       ...   
 2025-12-24  411.929993  412.630005  408.829987  412.250000   6718400   
 2025-12-26  416.739990  418.450012  414.750000  416.480011  10476200   
 2025-12-29  398.600006  403.760010  395.329987  403.660004  20679200   
 2025-12-30  398.890015  403.799988  398.559998  403.600006  10179000   
 2025-12-31  396.309998  400.130005  395.589996  398.600006  10194700   
 
 Price         Return  LogReturn  RollVol3

In [12]:
#creating correlation matrix for Return
assets_df = pd.DataFrame(columns=["Date", "GLD", "DXY", "TNX", "GSPC"])

In [13]:
for i in range(len(dfs["GLD"])):
    for asset in assets_df.columns:
        if asset == "Date":
            assets_df.loc[i, asset] = dfs["GLD"].index[i] #getting dates
        else:
            assets_df.loc[i, asset] = dfs[asset].iloc[i, -4] #getting Return values


In [14]:
#adjusting assets dataframe
assets_df.set_index(assets_df["Date"], inplace=True)
assets_df.drop(columns = "Date", inplace = True)
assets_df = assets_df.dropna()

/Users/mihailborisov/Desktop/portfolio/1(gold)/python_prep/gold_analysis/.venv/lib/python3.12/site-packages/pandas/core/indexes/base.py:7654: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [21]:
assets_df

,GLD,DXY,TNX,GSPC
Date,,,,
2019-01-03,0.009066,-0.005268,-0.04021,-0.024757
2019-01-04,-0.008086,-0.001246,0.041112,0.034336
2019-01-07,0.003458,-0.00499,0.00865,0.00701
2019-01-08,-0.002708,0.001985,0.012677,0.009695
2019-01-09,0.006418,-0.007091,0.004418,0.004098
...,...,...,...,...
2025-12-24,-0.004134,-0.003144,-0.007916,0.003221
2025-12-26,0.011677,-0.003561,0.0,-0.000304
2025-12-29,-0.043528,0.000408,-0.004836,-0.003492


In [17]:
#correlation matrix for Returns
corr_matrix = assets_df.corr()

In [18]:
corr_matrix

,GLD,DXY,TNX,GSPC
GLD,1.000000,-0.351521,-0.262228,0.105344
DXY,-0.351521,1.000000,0.205836,-0.166388
TNX,-0.262228,0.205836,1.000000,0.260962
GSPC,0.105344,-0.166388,0.260962,1.000000


In [19]:
corr_matrix.to_csv(f'/Users/mihailborisov/Desktop/gold/clean data/corr_matrix.csv', index=True, encoding='utf-8')